# **Problema dos Missionários e Canibais**

**Autor**: Matheus Agostinho

In [0]:
import time
import numpy as np

# Estado


In [0]:
class Estado:

    """ Representa um estado dentro de uma arvore de estados, aonde cada estado representa a configuração
    do problema, ou seja a posição de cada elemento. O estado contém a quantidade de missionário na margem esquerda
    (missionarios_esq), a quantidade de canibais na margem esquerda (canibais_esq), a quantidade de missionários
    na margem direita (missionarios_dir), a quantidade canibais na margem direita (canibais_dir), o lado da margem
    aonde o barco se encontra (lado_barco) e seu estado pai. """

    def __init__(self, missionarios_esq, canibais_esq, missionarios_dir, canibais_dir, lado_barco):
        """ Inicializa um estado com a quantidade de missionários e canibais em cada lado do rio,
        além da posição do barco. """

        self.missionarios_esq = missionarios_esq
        self.canibais_esq = canibais_esq
        self.missionarios_dir = missionarios_dir
        self.canibais_dir = canibais_dir

        self.lado_barco = lado_barco
        # Para o estado inicial o pai será nulo.
        self.pai = None

    def __eq__(self, outro):
        # Compara se um estado é igual ao outro pelo seus atributos.
        if isinstance(self, Estado) and isinstance(outro,Estado):
            # Se as classes forem diferentes a comparação não será válida.
            return self.missionarios_esq == outro.missionarios_esq \
            and self.canibais_esq == outro.canibais_esq \
            and self.missionarios_dir == outro.missionarios_dir \
            and self.canibais_dir == outro.canibais_dir \
            and self.lado_barco == outro.lado_barco
        return False

    def __str__(self):
        # Retorna um String contendo as informações do Estado.
        return '({}, {}, {}, {})'.format(self.missionarios_esq, self.canibais_esq, self.missionarios_dir, self.canibais_dir)

# Algoritmos de Busca


Busca em Largura


In [0]:
""" A busca em largura insere os nós adjacentes no final da
fronteira, pois usa uma estrutura de FIFO. """

class Busca_em_Largura:
    def coloca_fronteira(self, fronteira, adjacentes):
        fronteira =  fronteira + adjacentes
        return fronteira

Busca em Profundidade

In [0]:
""" A busca em largura insere os nós adjacentes no inicio da
fronteira, pois usa uma estrutura de LIFO. """

class Busca_em_Profundidade:
    def coloca_fronteira(self, fronteira, adjacentes):
        fronteira = adjacentes + fronteira
        return fronteira

Busca Gulosa

In [0]:
""" A busca gulosa insere inicialmente os nós no final, após
isso a lista será ordenada em ordem decrescente (melhores estados aparentes no início). """

class Busca_Gulosa:
    def coloca_fronteira(self, fronteira, adjacentes):
        #Função Heurística
        heuristica = lambda x : x.missionarios_esq + x.canibais_esq
        fronteira = fronteira + adjacentes
        #Ordena a lista com base na função heurística em ordem decrescente.
        fronteira.sort(key=heuristica, reverse=True)
        return fronteira

# Busca

In [0]:
class Busca:
    """ Representa todas operações que será feitas com os estados até chegar em uma 
    ou nenhuma solução. A busca contém o estado inicial que é a posição inicial
    dos missionários, canibais e do barco no escopo do problema, o algoritmo de busca
    que representa a estratégia de busca para alcançar o objetivo, além de sua
    de sua fronteira que será a estrutura de dados responsável por armazenar
    os estados que serão verificados e expandidos. """

    def __init__(self, estado_inicial, algoritmo_busca):
        """ Na inicialização é possível definir o estado inicial, além de
        seu algoritmo de busca que será um objeto da classe de busca. """

        self.estado_inicial = estado_inicial
        self.algoritmo_busca = algoritmo_busca
        
        self.fronteira = []
        self.fronteira.append(estado_inicial)

    def teste_objetivo(self, estado):
        """ O teste objetivo verifica se o estado atual é o estado esperado ou seja
        3 missionários e 3 canibais na margem esquerda do rio """

        missionarios = self.estado_inicial.missionarios_dir
        canibais = self.estado_inicial.canibais_dir

        return estado.canibais_esq == canibais \
        and estado.missionarios_esq == missionarios
        
    def valida_estado(self, estado):
        """ Verifica se o estado é valido, ou seja que não há um número
        negativo de missionários e canibais, e também se o estado obedece
        a restrição do problema, não tendo um número maior de canibais do 
        que de missionários. """

        if ((estado.missionarios_esq < 0) or (estado.missionarios_dir < 0)
        or (estado.canibais_esq < 0) or (estado.canibais_dir < 0)):
            return False
        
        return ((estado.missionarios_esq == 0 or estado.missionarios_esq >= estado.canibais_esq) and
                (estado.missionarios_dir == 0 or estado.missionarios_dir >= estado.canibais_dir))
        
    def gerar_sucessores(self, pai, visitados):
        """ Gera uma lista de estados sucessores com base na configuração
        do pai e da lista de nós visitados, os estados que não são válidos
        não são colocados na lista. """

        sucessores = []

        #Lista contendo um dicionário que indica o próximo movimentos
        movimentos = [
            {'missionarios': 2, 'canibais': 0},
            {'missionarios': 1, 'canibais': 0},
            {'missionarios': 1, 'canibais': 1},
            {'missionarios': 0, 'canibais': 1},
            {'missionarios': 0, 'canibais': 2},
        ]

        #Cada possível movimento presente no dicionário.
        for movimento in movimentos:
            #O barco está na margem direita.
            if pai.lado_barco:
                missionarios_dir = pai.missionarios_dir - movimento['missionarios']
                missionarios_esq = pai.missionarios_esq + movimento['missionarios']
                canibais_dir = pai.canibais_dir - movimento['canibais']
                canibais_esq = pai.canibais_esq + movimento['canibais']
            #O barco está na margem esquerda.
            else:
                missionarios_esq = pai.missionarios_esq - movimento['missionarios']
                missionarios_dir = pai.missionarios_dir + movimento['missionarios']
                canibais_esq = pai.canibais_esq - movimento['canibais']
                canibais_dir = pai.canibais_dir + movimento['canibais']
            
            estado = Estado(missionarios_esq, canibais_esq, missionarios_dir, canibais_dir, (pai.lado_barco + 1) % 2)
            estado.pai = pai

            """ Se o estado cumpre as restrições e não está na lista de estados visitados
            então ele será adicionado na fronteira e na lista de estados visitados. """
            if self.valida_estado(estado) and estado not in visitados:
                sucessores.append(estado)
                visitados.append(estado)

        return sucessores
                
    def gerar_solucao(self):
        """ Gera a solução, ou seja o caminho do estado inicial até o estado esperado do
        problema. Além da solução, é retornado o tempo gasto para calcular a solução
        e a quantidade de expansões que ele fez até achar a solução. """

        solucao = []
        visitados = []

        complexidade_tempo = time.time()
        complexidade_espaco = 0

        # Enquanto tiver algum estado na fronteira.
        while self.fronteira:
            # Retira sempre o primeiro estado na fronteira.
            estado = self.fronteira.pop(0)
            # Concatena a fronteira com os estados filhos expandidos do estado atual com base no algoritmo escolhido.
            self.fronteira = self.algoritmo_busca.coloca_fronteira(self.fronteira,self.gerar_sucessores(estado, visitados))
            complexidade_espaco += 1

            # Verifica se o estado é o estado objetivo.
            if self.teste_objetivo(estado):
                # Realiza o 'backtracking' do estado objetivo até o estado raiz (estado inicial).
                while estado.pai != None:
                    solucao.insert(0,estado)
                    estado = estado.pai
                solucao.insert(0, estado)
                break
        # Calcula o tempo necessário para calcular a solução.
        complexidade_tempo = time.time() - complexidade_tempo

        return solucao, complexidade_tempo, complexidade_espaco
    
    def traduz_solucao(self, solucao):
        """ Como a representação do problema é muito abstrata, é dificil aplicar
        a solução apresentada para um aplicação real do problema. Portanto a 
        solução será "traduzida" para uma representação mais agradável visualmente. """

        saida = []
        # Percorre a lista dos estados da solução.
        for n, estado in enumerate(solucao):
            if n < len(solucao) - 1:
                vizinho = solucao[n+1]
                
                # A ação realizada é calculada pela diferença absoluta entre os estados.
                me = abs(vizinho.missionarios_esq - estado.missionarios_esq)
                ce = abs(vizinho.canibais_esq - estado.canibais_esq)
                md = abs(vizinho.missionarios_dir - estado.missionarios_dir)
                cd = abs(vizinho.canibais_dir - estado.canibais_dir)

                lado = 'esquerda' if estado.lado_barco else 'direita'

                if (me == 2 or md == 2):
                    saida.append('Dois missionarios movem para margem {}'.format(lado))
                elif (ce == 2 or cd == 2):
                    saida.append('Dois canibais movem para margem {}'.format(lado))
                elif ((me == 1 and ce == 1) or (md == 1 and cd == 1)):
                    saida.append('Um missionario e um canibal movem para margem {}'.format(lado))
                elif me == 1 or md == 1:
                    saida.append('Um missionario move para margem {}'.format(lado))
                else:
                    saida.append('Um canibal move para margem {}'.format(lado))
        return saida

# Solução

In [11]:
# Estado inicial: Missionarios, Canibais, Missionarios, Canibais, Barco.
estado_inicial = Estado(0,0,3,3,1)
# Define o tipo de busca.
busca = Busca_em_Largura()
# Inicializa o problema.
problema = Busca(estado_inicial, busca)
# Gera a solução do problema.
solucao, tempo, espaco = problema.gerar_solucao()

for estado in solucao:
    print(estado, estado.lado_barco)

print()
for acao in problema.traduz_solucao(solucao):
    print(acao)


(0, 0, 2, 2) 1
(1, 1, 1, 1) 0
(0, 1, 2, 1) 1
(2, 1, 0, 1) 0
(1, 1, 1, 1) 1
(2, 2, 0, 0) 0

Um missionario e um canibal movem para margem esquerda
Um missionario move para margem direita
Dois missionarios movem para margem esquerda
Um missionario move para margem direita
Um missionario e um canibal movem para margem esquerda


# Análise


In [8]:
""" Para testar a performance de cada algoritmo de busca para o problema será realizado
várias iterações do problema, e será calculado a média de tempo para fins de análise. """

# Número de iterações.
n = 1000

algoritmos = ['Busca em Largura', 'Busca em Profundidade', 'Busca Gulosa']
valores = {}

estado_inicial = Estado(0,0,3,3,1)
# Realiza a busca n vezes.
for i in range(n):

    for algoritmo in algoritmos:
        if algoritmo == 'Busca em Largura':
            busca = Busca_em_Largura()
        elif algoritmo == 'Busca em Profundidade':
            busca = Busca_em_Profundidade()
        elif algoritmo == 'Busca Gulosa':
            busca = Busca_Gulosa()

                
        problema = Busca(estado_inicial, busca)    
        solucao, tempo, espaco = problema.gerar_solucao()

        # Retorna uma lista vazia, caso não ache algum elemento
        valores[algoritmo] = valores.get(algoritmo, []) + [(len(solucao) - 1, tempo, espaco)]
        # O campo valores do algoritmo armazena o tamanho da solução, o tempo e o espaço.

#Dicionário para armazenar os resultados.
comp = {}
# Para cada algoritmo será gerado um resultado.
for algoritmo in algoritmos:
    print('Algoritmo:', algoritmo,'\n')

    # Número de passos do problema.
    passos = valores[algoritmo][0][0]
    # Média de tempo do problema.
    media_tempo = np.mean([n[1] for n in valores[algoritmo]])
    # Calcula o desvio padrão do  tempo do problema.
    desvio_tempo = np.std([n[1] for n in valores[algoritmo]])
    # Calcula a quantidade de vezes que foi necessário expandir um estado.
    espaco = valores[algoritmo][0][2]

    #Armazena o resultado de cada algoritmo no dicionário.
    comp[algoritmo] = [passos, media_tempo, desvio_tempo, espaco]
    

    print('Número de passos: [{}]'.format(comp[algoritmo][0]))
    print('Média do tempo de processamento: [{} s]'.format(comp[algoritmo][1]))
    print('Desvio padrão do tempo de processamento: [{} s]'.format(comp[algoritmo][2]))
    print('Número de expansões necessárias para alcançar a solução: [{}]'.format(comp[algoritmo][3]))
    print('\n', '-' * 30)

#Ordena os valores pelas métricas apresentadas anteriormente.
comp = sorted(comp.items(), key=lambda item: item[1])

#Imprime a lista dos algoritmos ordenados pela sua eficiência.
for i in comp:
    print(i[0])



Algoritmo: Busca em Largura 

Número de passos: [5]
Média do tempo de processamento: [0.0001635868549346924 s]
Desvio padrão do tempo de processamento: [4.160665932254449e-05 s]
Número de expansões necessárias para alcançar a solução: [11]

 ------------------------------
Algoritmo: Busca em Profundidade 

Número de passos: [5]
Média do tempo de processamento: [0.0001256237030029297 s]
Desvio padrão do tempo de processamento: [2.681126777175849e-05 s]
Número de expansões necessárias para alcançar a solução: [8]

 ------------------------------
Algoritmo: Busca Gulosa 

Número de passos: [5]
Média do tempo de processamento: [0.0001462552547454834 s]
Desvio padrão do tempo de processamento: [3.3676624111580634e-05 s]
Número de expansões necessárias para alcançar a solução: [9]

 ------------------------------
Busca em Profundidade
Busca Gulosa
Busca em Largura
